In [11]:
import numpy as np
import cupy as cp

#### Kernels are functions that will be run on the GPU. CuPy provides easy ways to define three types of CUDA kernels: elementwise kernels, reduction kernels and raw kernels

## Elementwise Kernels

An element wise kerenel has 4 parts :
1. an input argument list:
2. an output argument list
3. loop body : defines the operation on each element of the argument
4. kernel name

In [12]:
element_diff = cp.ElementwiseKernel('float32 x, float32 y', 
                                    'float32 z', 
                                    'z = (x - y)', 
                                    'element_diff')



In [13]:
x = cp.array([1, 2, 3], dtype=np.float32)
y = cp.array([4, 5, 6], dtype=np.float32)

In [14]:
z = element_diff(x, y)
print(z)

[-3. -3. -3.]


In [15]:
z = cp.empty((1, 3), dtype=np.float32)
element_diff(x, y, z)
print(z)

[[-3. -3. -3.]]


#### What happens when the ndarray is of different dimension?

#### Type can be inffered from the arguments

In [56]:
element_diff = cp.ElementwiseKernel('T x, T y', 
                                    'T z', 'z = (x - y)', 
                                    'element_diff')

In [57]:
x = cp.array([1, 2, 3], dtype=np.float32)
y = cp.array([4, 5, 6], dtype=np.float32)

z = element_diff(x, y)
print(z)



[-3. -3. -3.]


In [58]:
x = cp.array([1, 2, 3], dtype=np.int32)
y = cp.array([4, 5, 6], dtype=np.int32)

z = element_diff(x, y)
print(z)

[-3 -3 -3]


##### We can also write a kernel with manual indexing for some arguments by telling ElementwiseKernel class to use manual indexing. This is done by adding the ``raw`` keyword preceding the type specifier

In [54]:
# i indicates the index within the loop
# _ind.size() indicates total number of elements to apply the elementwise operation

element_sum = cp.ElementwiseKernel('T x, raw T y', 
                                   'T z', 'z = x + y[_ind.size() - i - 1]', 
                                   'element_sum')


In [55]:
x = cp.array([1, 2, 3], dtype=np.float32)
y = cp.array([4, 5, 6], dtype=np.float32)

z = element_sum(x, y)
print(z)

[7. 7. 7.]


## Reduction kernels

A reduction kernel has 4 parts :
1. Identity value: This value is used for the initial value of reduction.
2. Mapping expression: It is used for the pre-processing of each element to be reduced.
3. Reduction expression: It is an operator to reduce the multiple mapped values. The special variables a and b are used for its operands.
4. Post mapping expression: It is used to transform the resulting reduced values. The special variable a is used as its input. Output should be written to the output parameter.

In [43]:
reduction_kernel = cp.ReductionKernel(
    'T x', # input param
    'T y',  # output param
    'x',  # map
    'a + b',  # reduce
    'y = a',  # post-reduction map
    '0',  # identity value
    'reduction_kernel'  # kernel name
)

In [44]:
x = cp.arange(10, dtype=np.float32).reshape(1, 10)
print(x)

[[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]]


In [45]:
reduction_kernel(x, axis=1)

array([45.], dtype=float32)

In [46]:
x = cp.arange(10, dtype=np.float32).reshape(2, 5)
print(x)

[[0. 1. 2. 3. 4.]
 [5. 6. 7. 8. 9.]]


In [47]:
reduction_kernel(x, axis=1)

array([10., 35.], dtype=float32)

#### TODO: Change the above function to find the sum of squares of each element in the array
That is, If the array is [2, 3, 4, 5], find 2^2 + 3^2 + 4^2 + 5^2 

## Raw kernels

In [51]:

add_kernel = cp.RawKernel(r'''
    extern "C" __global__
    void my_add(const float* x1, const float* x2, float* y) 
    {
        int tid = blockDim.x * blockIdx.x + threadIdx.x;
        y[tid] = x1[tid] + x2[tid];
    }''', 'my_add')



In [52]:
x = cp.arange(25, dtype=cp.float32).reshape(5, 5)
y = cp.arange(25, dtype=cp.float32).reshape(5, 5)
z = cp.zeros((5, 5), dtype=cp.float32)



In [53]:
# When calling a raw kernel ypu have to specify  
# how threads are grouped (grids and blocks)
# https://developer.nvidia.com/blog/cuda-refresher-cuda-programming-model/

add_kernel((5,), (5,), (x, y, z))
print(z)

[[ 0.  2.  4.  6.  8.]
 [10. 12. 14. 16. 18.]
 [20. 22. 24. 26. 28.]
 [30. 32. 34. 36. 38.]
 [40. 42. 44. 46. 48.]]
